In [1]:
import pandas as pd
import json

with open('data.ipynb', 'r') as file:
    data = json.load(file)

df = pd.json_normalize(data, sep='_')

print(df.head())


        _id appointmentId  phoneNumber patientDetails__id  \
0  T6hf3rb5     40d2-9c9f  96686896670          T6hb630b3   
1  T65g3rb5      40dbtc9f   9496368916          T6h33b300   
2  T7g6Srb5      g3Wt5c9f   7787204833           TjhB4373   
3  94bg8W8d      fb6-a535   9376756879            6df4R5b   
4  T6h8H56c       67h7KL9   5267384241           lK9hy06a   

  patientDetails_firstName patientDetails_lastName patientDetails_emailId  \
0                      Css                                                  
1                   Lokesh                                                  
2                    Shila                     Das                          
3                  Bhavika             Ben Panchal                          
4               Raghu Viju                                                  

  patientDetails_gender patientDetails_alternateContact  \
0                                                         
1                     M                             

In [9]:
import pandas as pd
import json
with open('data.ipynb', 'r') as file:
    data = json.load(file)

df = pd.json_normalize(data, sep='_')

patient_data = df[['patientDetails_firstName', 'patientDetails_lastName', 'patientDetails_birthDate']]


patient_data.replace('', pd.NA, inplace=True)

missing_percentage = patient_data.isna().mean() * 100

first_name_missing = round(missing_percentage['patientDetails_firstName'], 2)
last_name_missing = round(missing_percentage['patientDetails_lastName'], 2)
dob_missing = round(missing_percentage['patientDetails_birthDate'], 2)

result = f"{first_name_missing}, {last_name_missing}, {dob_missing}"
result


/var/folders/bm/gdcfqs196fl9vf7w95gf55k40000gn/T/ipykernel_1809/1403471620.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_data.replace('', pd.NA, inplace=True)


'0.0, 70.97, 32.26'

In [10]:

gender_data = df[['patientDetails_gender']]

gender_data.replace('', pd.NA, inplace=True)

mode_gender = gender_data['patientDetails_gender'].mode()[0]

gender_data['patientDetails_gender'].fillna(mode_gender, inplace=True)

female_percentage = (gender_data['patientDetails_gender'] == 'F').mean() * 100
female_percentage_rounded = round(female_percentage, 2)

female_percentage_rounded


/var/folders/bm/gdcfqs196fl9vf7w95gf55k40000gn/T/ipykernel_1809/3881381252.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_data.replace('', pd.NA, inplace=True)
/var/folders/bm/gdcfqs196fl9vf7w95gf55k40000gn/T/ipykernel_1809/3881381252.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_data['patientDetails_gender'].fillna(mode_gender, inplace=True)


32.26

In [15]:
current_date = pd.Timestamp.now().normalize()

df['patientDetails_birthDate'] = pd.to_datetime(df['patientDetails_birthDate'], errors='coerce').dt.tz_localize(None)

df['age'] = (current_date - df['patientDetails_birthDate']).dt.days // 365

choices = ['Child', 'Teen', 'Adult', 'Senior']
df['ageGroup'] = pd.cut(df['age'], bins=[-1, 12, 19, 59, float('inf')], labels=choices)

adult_count = df['ageGroup'].value_counts().get('Adult', 0)
adult_count



21

In [16]:
df['number_of_medicines'] = df['consultationData_medicines'].apply(lambda x: len(x))

average_medicines = df['number_of_medicines'].mean()
average_medicines_rounded = round(average_medicines, 2)

average_medicines_rounded


2.13

In [17]:
medicines_data = df[['consultationData_medicines']].explode('consultationData_medicines').reset_index(drop=True)

medicines_normalized = pd.json_normalize(medicines_data['consultationData_medicines'])
medicines_data = pd.concat([medicines_data.drop(columns=['consultationData_medicines']), medicines_normalized], axis=1)

medicine_frequency = medicines_data['medicineName'].value_counts()

third_most_frequent_medicine = medicine_frequency.index[2]
third_most_frequent_medicine


'C'

In [18]:
active_inactive_counts = medicines_data['isActive'].value_counts(normalize=True) * 100

active_percentage = round(active_inactive_counts[True], 2)
inactive_percentage = round(active_inactive_counts[False], 2)

result = f"{active_percentage}, {inactive_percentage}"
result


'69.7, 30.3'

In [19]:
def is_valid_indian_phone_number(phone):
    if phone.startswith('+91'):
        phone = phone[3:]
    elif phone.startswith('91'):
        phone = phone[2:]

    if phone.isdigit() and len(phone) == 10 and 6000000000 <= int(phone) <= 9999999999:
        return True
    else:
        return False
df['isValidMobile'] = df['phoneNumber'].apply(is_valid_indian_phone_number)

valid_phone_numbers_count = df['isValidMobile'].sum()

valid_phone_numbers_count


18

In [22]:
combined_data = pd.concat([df['age'], df['number_of_medicines']], axis=1).dropna()

correlation = combined_data.corr(method='pearson').iloc[0, 1]
correlation_rounded = round(correlation, 2)

correlation_rounded


-0.22